# Automating PubMed Search with LangChain Agent 🔎

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
<p style="margin-right:10px;">
    <img  height="200px" src="https://images.contentstack.io/v3/assets/bltf2fca5bf44f5e817/blt34d9fdb635976e4a/669e80a79fecd86c50d59f6d/Lang_Square.png"> 
</p>

</div>

## Description :

#### This code automates the process of querying PubMed, a database of biomedical literature, using an AI-powered LangChain agent. The agent is designed to find cures for diseases by retrieving and summarizing scientific papers from PubMed.

## Features

### AI Model Initialization
- Utilizes OpenAI's GPT-4 (`ChatOpenAI`) for processing and understanding natural language queries.

### Tool Integration
- Integrates the `PubmedQueryRun` tool for querying PubMed.

### Custom Agent Prompt
- Defines the agent's role and behavior:
  - Acts as a virtual assistant ("SV") specializing in disease cures.
  - Cites references for facts in bracketed format.

### Question Answering
- Processes a predefined question: **"How can COVID-19 be treated?"**
- Uses the PubMed tool to fetch relevant information and generate a fact-based response.

### Agent Execution
- Runs the agent using `AgentExecutor`, which manages tool interaction and verbose output for debugging.

## Purpose
This script simplifies the process of retrieving scientific insights from PubMed, making it a powerful tool for researchers, healthcare professionals, and students seeking evidence-based answers to medical questions.




## Step 1: Environment Setup and Requirements Installation Script

This script sets up the environment for the notebook by installing necessary dependencies and checking for required environment variables. It installs the dependencies from a `requirements.txt file`, retries up to three times in case of failure, and ensures that essential environment variables are set. If any variable is missing, the script exits and prompts the user to set it. Once the setup is complete, a success message is displayed.

- Import required libraries (os, dotenv, IPython.display for clearing output).

- Initialize variables for retrying and environment setup.

- Define install_requirements() to install dependencies from requirements.txt,
 with retry logic.

- Define setup_env() to check and load required environment variables.

- Execute install_requirements() and setup_env() to finalize the setup process.

In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True)

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2 - Setting up the LangChain Agent and PubMed Tool 

#### This code automates querying PubMed for biomedical research using a LangChain agent integrated with GPT-4.

1. **Import Required Libraries**  
   Import `AgentExecutor`, `PubmedQueryRun`, and `ChatOpenAI` to set up the LangChain agent and PubMed tool.

2. **Initialize Language Model**  
   Create an instance of GPT-4 (`ChatOpenAI`) to process natural language queries.

3. **Set Up PubMed Query Tool**  
   Initialize `PubmedQueryRun` to enable querying PubMed.

4. **Define Tools**  
   Add the PubMed tool to the tools list.

5. **Create Agent Prompt**  
   Define the agent's role and communication behavior using `ChatPromptTemplate`.

6. **Set User Query**  
   Provide the question, `"How can COVID-19 be treated?"`.

7. **Create Agent**  
   Combine the LLM, tools, and prompt into a LangChain agent using `create_tool_calling_agent`.

8. **Execute the Agent**  
   Use `AgentExecutor` to process the query and retrieve results from PubMed.

9. **Print the Output**  
   Display the agent's response with references.


In [ ]:
## Simple PubMed Tool with LangChain Agent

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o")

pubmed_tool = PubmedQueryRun()

tools = [pubmed_tool]

messages = [
    (
        "ai",
        "You are an AI called SV that's is designed to find cures for diseases. You have access to a database of scientific papers and can ask for more information if needed.",
    ),
    (
        "user",
        "{question}. Provide the names of references in the bracket whenever you mention a fact.",
    ),
    ("placeholder", "{agent_scratchpad}"),
]

question = "How can COVID-19 be treated?"

prompt = ChatPromptTemplate.from_messages(messages)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"question": question})

print(result.get("output"))

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>
